In [1]:
import pandas as pd

In [2]:
df_customers=pd.read_csv("C:/Users/Lenovo/Downloads/Customers.csv",encoding="latin-1")

In [3]:
df_customers.head()

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,7/3/1939
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,9/27/1979
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,5/26/1947
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,9/17/1957
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,11/19/1965


In [4]:
df_customers.isnull().sum()

CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64

In [8]:
def null_value_imputer(df_customers):
    for i in df_customers.columns:
        # Drop columns with 80% or more missing values
        if (df_customers[i].isnull().sum() / len(df_customers)) * 100 >= 80:
            df_customers.drop(i, axis=1, inplace=True)
        # Impute missing values in numeric columns
        elif df_customers[i].dtype in ['int64', 'float64']:
            # Check the skewness of the column
            if -0.4 < df_customers[i].skew() < 0.4:
                # If skewness is low, fill missing values with the mean
                df_customers[i].fillna(df_customers[i].mean(), inplace=True)
            else:
                # If skewness is high, fill missing values with the median
                df_customers[i].fillna(df_customers[i].median(), inplace=True)
        else:
            # For categorical columns, fill missing values with the mode
            df_customers[i].fillna(df_customers[i].mode()[0], inplace=True)
    
    # Print the number of null values after imputation
    print("Null Values after imputation:\n", df_customers.isnull().sum())
    print(df_customers.isnull().sum())

In [9]:
null_value_imputer(df_customers)

Null Values after imputation:
 CustomerKey    0
Gender         0
Name           0
City           0
State Code     0
State          0
Zip Code       0
Country        0
Continent      0
Birthday       0
dtype: int64
CustomerKey    0
Gender         0
Name           0
City           0
State Code     0
State          0
Zip Code       0
Country        0
Continent      0
Birthday       0
dtype: int64


In [10]:
df_customers.isnull().sum()

CustomerKey    0
Gender         0
Name           0
City           0
State Code     0
State          0
Zip Code       0
Country        0
Continent      0
Birthday       0
dtype: int64

In [11]:
df_customers.duplicated().sum()

0

In [12]:
df_customers.rename(columns={'State Code': 'State_Code'}, inplace=True)
df_customers.rename(columns={'Zip Code': 'Zip_Code'}, inplace=True)
df_customers.rename(columns={'CustomerKey':'Customer_Key'},inplace=True)

In [13]:
df_customers.head()

,Customer_Key,Gender,Name,City,State_Code,State,Zip_Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,7/3/1939
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,9/27/1979
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,5/26/1947
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,9/17/1957
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,11/19/1965


In [14]:
df_customers.dtypes

Customer_Key     int64
Gender          object
Name            object
City            object
State_Code      object
State           object
Zip_Code        object
Country         object
Continent       object
Birthday        object
dtype: object

In [15]:
df_customers['Birthday'] = pd.to_datetime(df_customers['Birthday'], errors='coerce')
df_customers['Birthday'] = df_customers['Birthday'].dt.strftime('%Y-%m-%d')

In [16]:
df_customers.head()

,Customer_Key,Gender,Name,City,State_Code,State,Zip_Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,1939-07-03
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,1947-05-26
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,1957-09-17
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,1965-11-19


In [17]:
df_customers.dtypes

Customer_Key     int64
Gender          object
Name            object
City            object
State_Code      object
State           object
Zip_Code        object
Country         object
Continent       object
Birthday        object
dtype: object

In [18]:
df_customers.to_csv('customers_data.csv', index=False)

In [20]:
import pymysql

In [21]:
myconnection = pymysql.connect(host='127.0.0.1',user='root',password='root')

In [22]:
myconnection.cursor().execute("create database data")

1

In [23]:
a=",".join(f"{i} {j}"
for i,j in zip(df_customers.columns,df_customers.dtypes)).replace("float64","float").replace("category","text").replace("int64","int").replace("object","text")

In [24]:
f"create table customers_data ({a})"

'create table customers_data (Customer_Key int,Gender text,Name text,City text,State_Code text,State text,Zip_Code text,Country text,Continent text,Birthday text)'

In [25]:
myconnection.cursor().execute(f"create table data.customers_data ({a})")

0

In [26]:
len(df_customers)

15266

In [28]:
sql = "insert into data.customers_data values"
for i in range(len(df_customers)):
    myconnection.cursor().execute(f"{sql} {tuple(df_customers.iloc[i])}")
    myconnection.commit()

In [29]:
df_customers.head()

,Customer_Key,Gender,Name,City,State_Code,State,Zip_Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,1939-07-03
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,1947-05-26
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,1957-09-17
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,1965-11-19


In [ ]:
########################################SALES###############################

In [30]:
df_sales=pd.read_csv("C:/Users/Lenovo/Downloads/Sales.csv",encoding="latin-1")

In [31]:
df_sales.head()

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,1/1/2016,NaN,265598,10,1304,1,CAD
1,366001,1,1/1/2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,1/1/2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,1/1/2016,1/12/2016,266019,0,1106,7,CAD
4,366002,2,1/1/2016,1/12/2016,266019,0,373,1,CAD


In [32]:
df_sales.isnull().sum()

Order Number         0
Line Item            0
Order Date           0
Delivery Date    49719
CustomerKey          0
StoreKey             0
ProductKey           0
Quantity             0
Currency Code        0
dtype: int64

In [33]:
def null_value_imputer(df_sales):
    for i in df_sales.columns:
        # Drop columns with 80% or more missing values
        if (df_sales[i].isnull().sum() / len(df_sales)) * 100 >= 80:
            df_sales.drop(i, axis=1, inplace=True)
        # Impute missing values in numeric columns
        elif df_sales[i].dtype in ['int64', 'float64']:
            # Check the skewness of the column
            if -0.4 < df_sales[i].skew() < 0.4:
                # If skewness is low, fill missing values with the mean
                df_sales[i].fillna(df_sales[i].mean(), inplace=True)
            else:
                # If skewness is high, fill missing values with the median
                df_sales[i].fillna(df_sales[i].median(), inplace=True)
        else:
            # For categorical columns, fill missing values with the mode
            df_sales[i].fillna(df_sales[i].mode()[0], inplace=True)
    
    # Print the number of null values after imputation
    print("Null Values after imputation:\n", df_sales.isnull().sum())
    print(df_sales.isnull().sum())


In [34]:
null_value_imputer(df_sales)

Null Values after imputation:
 Order Number     0
Line Item        0
Order Date       0
Delivery Date    0
CustomerKey      0
StoreKey         0
ProductKey       0
Quantity         0
Currency Code    0
dtype: int64
Order Number     0
Line Item        0
Order Date       0
Delivery Date    0
CustomerKey      0
StoreKey         0
ProductKey       0
Quantity         0
Currency Code    0
dtype: int64


In [35]:
df_sales.duplicated().sum()

0

In [36]:
df_sales.rename(columns={'Order Number': 'Order_Number'}, inplace=True)
df_sales.rename(columns={'Line Item': 'Line_Item'}, inplace=True)
df_sales.rename(columns={'Order Date': 'Order_Date'}, inplace=True)
df_sales.rename(columns={'Delivery Date': 'Delivery_Date'}, inplace=True)
df_sales.rename(columns={'Currency Code': 'Currency_Code'}, inplace=True)
df_sales.rename(columns={'CustomerKey': 'Customer_Key'}, inplace=True)
df_sales.rename(columns={'StoreKey': 'Store_Key'}, inplace=True)
df_sales.rename(columns={'ProductKey': 'Product_Key'}, inplace=True)

In [37]:
df_sales.head()

,Order_Number,Line_Item,Order_Date,Delivery_Date,Customer_Key,Store_Key,Product_Key,Quantity,Currency_Code
0,366000,1,1/1/2016,1/6/2020,265598,10,1304,1,CAD
1,366001,1,1/1/2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,1/1/2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,1/1/2016,1/12/2016,266019,0,1106,7,CAD
4,366002,2,1/1/2016,1/12/2016,266019,0,373,1,CAD


In [38]:
df_sales.dtypes

Order_Number      int64
Line_Item         int64
Order_Date       object
Delivery_Date    object
Customer_Key      int64
Store_Key         int64
Product_Key       int64
Quantity          int64
Currency_Code    object
dtype: object

In [39]:
df_sales['Order_Date'] = pd.to_datetime(df_sales['Order_Date'], errors='coerce')
df_sales['Order_Date'] = df_sales['Order_Date'].dt.strftime('%Y-%m-%d')


df_sales['Delivery_Date'] = pd.to_datetime(df_sales['Delivery_Date'], errors='coerce')
df_sales['Delivery_Date'] = df_sales['Delivery_Date'].dt.strftime('%Y-%m-%d')

In [40]:
df_sales.head()

,Order_Number,Line_Item,Order_Date,Delivery_Date,Customer_Key,Store_Key,Product_Key,Quantity,Currency_Code
0,366000,1,2016-01-01,2020-01-06,265598,10,1304,1,CAD
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD


In [41]:
df_sales.dtypes

Order_Number      int64
Line_Item         int64
Order_Date       object
Delivery_Date    object
Customer_Key      int64
Store_Key         int64
Product_Key       int64
Quantity          int64
Currency_Code    object
dtype: object

In [42]:
df_sales.to_csv('sales_data.csv', index=False)

In [43]:
import pymysql

In [44]:
myconnection = pymysql.connect(host='127.0.0.1',user='root',password='root')

In [45]:
a=",".join(f"{i} {j}"
for i,j in zip(df_sales.columns,df_sales.dtypes)).replace("float64","float").replace("category","text").replace("int64","int").replace("object","text")

In [47]:
f"create table sales_data ({a})"

'create table sales_data (Order_Number int,Line_Item int,Order_Date text,Delivery_Date text,Customer_Key int,Store_Key int,Product_Key int,Quantity int,Currency_Code text)'

In [48]:
myconnection.cursor().execute(f"create table data.sales_data ({a})")

0

In [49]:
len(df_sales)

62884

In [50]:
sql = "insert into data.sales_data values"
for i in range(len(df_sales)):
    myconnection.cursor().execute(f"{sql} {tuple(df_sales.iloc[i])}")
    myconnection.commit()

In [51]:
df_sales.head()

,Order_Number,Line_Item,Order_Date,Delivery_Date,Customer_Key,Store_Key,Product_Key,Quantity,Currency_Code
0,366000,1,2016-01-01,2020-01-06,265598,10,1304,1,CAD
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD


In [ ]:
###########################################PRODUCTS##############################################

In [58]:
import pandas as pd

In [59]:
df_products=pd.read_csv("C:/Users/Lenovo/Downloads/Products.csv",encoding="latin-1")

In [60]:
df_products.head()

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,$6.62,$12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,$6.62,$12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,$7.40,$14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,$11.00,$21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,$11.00,$21.57,101,MP4&MP3,1,Audio


In [61]:
df_products.isnull().sum()

ProductKey        0
Product Name      0
Brand             0
Color             0
Unit Cost USD     0
Unit Price USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
dtype: int64

In [62]:
def null_value_imputer(df_products):
    for i in df_products.columns:
        # Drop columns with 80% or more missing values
        if (df_products[i].isnull().sum() / len(df_products)) * 100 >= 80:
            df_products.drop(i, axis=1, inplace=True)
        # Impute missing values in numeric columns
        elif df_products[i].dtype in ['int64', 'float64']:
            # Check the skewness of the column
            if -0.4 < df_products[i].skew() < 0.4:
                # If skewness is low, fill missing values with the mean
                df_products[i].fillna(df_products[i].mean(), inplace=True)
            else:
                # If skewness is high, fill missing values with the median
                df_products[i].fillna(df_products[i].median(), inplace=True)
        else:
            # For categorical columns, fill missing values with the mode
            df_products[i].fillna(df_products[i].mode()[0], inplace=True)
    
    # Print the number of null values after imputation
    print("Null Values after imputation:\n", df_products.isnull().sum())
    print(df_products.isnull().sum())


In [63]:
null_value_imputer(df_products)

Null Values after imputation:
 ProductKey        0
Product Name      0
Brand             0
Color             0
Unit Cost USD     0
Unit Price USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
dtype: int64
ProductKey        0
Product Name      0
Brand             0
Color             0
Unit Cost USD     0
Unit Price USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
dtype: int64


In [64]:
df_products.duplicated().sum()

0

In [65]:
df_products.rename(columns={'ProductKey': 'Product_Key'}, inplace=True)
df_products.rename(columns={'Product Name': 'Product_Name'}, inplace=True)
df_products.rename(columns={'Unit Cost USD': 'Unit_Cost_USD'}, inplace=True)
df_products.rename(columns={'Unit Price USD': 'Unit_Price_USD'}, inplace=True)
df_products.rename(columns={'SubcategoryKey': 'Sub_category_Key'}, inplace=True)
df_products.rename(columns={'Subcategory': 'Sub_category'}, inplace=True)
df_products.rename(columns={'CategoryKey': 'Category_Key'}, inplace=True)


In [67]:
df_products.head()

,Product_Key,Product_Name,Brand,Color,Unit_Cost_USD,Unit_Price_USD,Sub_category_Key,Sub_category,Category_Key,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,$6.62,$12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,$6.62,$12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,$7.40,$14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,$11.00,$21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,$11.00,$21.57,101,MP4&MP3,1,Audio


In [68]:
df_products.dtypes

Product_Key          int64
Product_Name        object
Brand               object
Color               object
Unit_Cost_USD       object
Unit_Price_USD      object
Sub_category_Key     int64
Sub_category        object
Category_Key         int64
Category            object
dtype: object

In [69]:
df_products.to_csv('products_data.csv', index=False)

In [70]:
myconnection = pymysql.connect(host='127.0.0.1',user='root',password='root')

In [71]:
a=",".join(f"{i} {j}"
for i,j in zip(df_products.columns,df_products.dtypes)).replace("float64","float").replace("category","text").replace("int64","int").replace("object","text")

In [72]:
f"create table products_data ({a})"

'create table products_data (Product_Key int,Product_Name text,Brand text,Color text,Unit_Cost_USD text,Unit_Price_USD text,Sub_text_Key int,Sub_text text,Category_Key int,Category text)'

In [73]:
myconnection.cursor().execute(f"create table data.products_data ({a})")

0

In [74]:
len(df_products)

2517

In [75]:
sql = "insert into data.products_data values"
for i in range(len(df_products)):
    myconnection.cursor().execute(f"{sql} {tuple(df_products.iloc[i])}")
    myconnection.commit()

In [76]:
df_products.head()

,Product_Key,Product_Name,Brand,Color,Unit_Cost_USD,Unit_Price_USD,Sub_category_Key,Sub_category,Category_Key,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,$6.62,$12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,$6.62,$12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,$7.40,$14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,$11.00,$21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,$11.00,$21.57,101,MP4&MP3,1,Audio


In [ ]:
####################################STORES##########################################

In [77]:
import pandas as pd

In [77]:
df_stores=pd.read_csv("C:/Users/Lenovo/Downloads/Stores.csv",encoding="latin-1")

In [78]:
df_stores.head()

,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015


In [79]:
df_stores.isnull().sum()

StoreKey         0
Country          0
State            0
Square Meters    1
Open Date        0
dtype: int64

In [80]:
def null_value_imputer(df_stores):
    for i in df_stores.columns:
        # Drop columns with 80% or more missing values
        if (df_stores[i].isnull().sum() / len(df_stores)) * 100 >= 80:
            df_stores.drop(i, axis=1, inplace=True)
        # Impute missing values in numeric columns
        elif df_stores[i].dtype in ['int64', 'float64']:
            # Check the skewness of the column
            if -0.4 < df_stores[i].skew() < 0.4:
                # If skewness is low, fill missing values with the mean
                df_stores[i].fillna(df_stores[i].mean(), inplace=True)
            else:
                # If skewness is high, fill missing values with the median
                df_stores[i].fillna(df_stores[i].median(), inplace=True)
        else:
            # For categorical columns, fill missing values with the mode
            df_stores[i].fillna(df_stores[i].mode()[0], inplace=True)
    
    # Print the number of null values after imputation
    print("Null Values after imputation:\n", df_stores.isnull().sum())
    print(df_stores.isnull().sum())

In [81]:
null_value_imputer(df_stores)

Null Values after imputation:
 StoreKey         0
Country          0
State            0
Square Meters    0
Open Date        0
dtype: int64
StoreKey         0
Country          0
State            0
Square Meters    0
Open Date        0
dtype: int64


In [82]:
df_stores.duplicated().sum()

0

In [83]:
df_stores.rename(columns={'StoreKey': 'Store_Key'}, inplace=True)
df_stores.rename(columns={'Square Meters': 'Square_Meters'}, inplace=True)
df_stores.rename(columns={'Open Date': 'Open_Date'}, inplace=True)

In [85]:
df_stores.dtypes

Store_Key          int64
Country           object
State             object
Square_Meters    float64
Open_Date         object
dtype: object

In [86]:
df_stores['Open_Date'] = pd.to_datetime(df_stores['Open_Date'], errors='coerce')
df_stores['Open_Date'] = df_stores['Open_Date'].dt.strftime('%Y-%m-%d')

In [87]:
df_stores.head()

,Store_Key,Country,State,Square_Meters,Open_Date
0,1,Australia,Australian Capital Territory,595.0,2008-01-01
1,2,Australia,Northern Territory,665.0,2008-01-12
2,3,Australia,South Australia,2000.0,2012-01-07
3,4,Australia,Tasmania,2000.0,2010-01-01
4,5,Australia,Victoria,2000.0,2015-12-09


In [88]:
df_stores.to_csv('stores_data.csv', index=False)

In [89]:
import pymysql

In [90]:
myconnection = pymysql.connect(host='127.0.0.1',user='root',password='root')

In [91]:
a=",".join(f"{i} {j}"
for i,j in zip(df_stores.columns,df_stores.dtypes)).replace("float64","float").replace("category","text").replace("int64","int").replace("object","text")

In [92]:
f"create table stores_data ({a})"

'create table stores_data (Store_Key int,Country text,State text,Square_Meters float,Open_Date text)'

In [93]:
myconnection.cursor().execute(f"create table data.stores_data ({a})")

0

In [94]:
len(df_stores)

67

In [95]:
sql = "insert into data.stores_data values"
for i in range(len(df_stores)):
    myconnection.cursor().execute(f"{sql} {tuple(df_stores.iloc[i])}")
    myconnection.commit()

In [96]:
df_stores.head()

,Store_Key,Country,State,Square_Meters,Open_Date
0,1,Australia,Australian Capital Territory,595.0,2008-01-01
1,2,Australia,Northern Territory,665.0,2008-01-12
2,3,Australia,South Australia,2000.0,2012-01-07
3,4,Australia,Tasmania,2000.0,2010-01-01
4,5,Australia,Victoria,2000.0,2015-12-09


In [ ]:
#########################################################EXCHANGE_RATES#####################################

In [97]:
import pandas as pd

In [98]:
df_exchange_rates=pd.read_csv("C:/Users/Lenovo/Downloads/Exchange_Rates.csv",encoding="latin-1")

In [99]:
df_exchange_rates.isnull().sum()

Date        0
Currency    0
Exchange    0
dtype: int64

In [100]:
def null_value_imputer(df_exchange_rates):
    for i in df_exchange_rates.columns:
        # Drop columns with 80% or more missing values
        if (df_exchange_rates[i].isnull().sum() / len(df_exchange_rates)) * 100 >= 80:
            df_exchange_rates.drop(i, axis=1, inplace=True)
        # Impute missing values in numeric columns
        elif df_exchange_rates[i].dtype in ['int64', 'float64']:
            # Check the skewness of the column
            if -0.4 < df_exchange_rates[i].skew() < 0.4:
                # If skewness is low, fill missing values with the mean
                df_exchange_rates[i].fillna(df_exchange_rates[i].mean(), inplace=True)
            else:
                # If skewness is high, fill missing values with the median
                df_exchange_rates[i].fillna(df_exchange_rates[i].median(), inplace=True)
        else:
            # For categorical columns, fill missing values with the mode
            df_exchange_rates[i].fillna(df_exchange_rates[i].mode()[0], inplace=True)
    
    # Print the number of null values after imputation
    print("Null Values after imputation:\n", df_exchange_rates.isnull().sum())
    print(df_exchange_rates.isnull().sum())


In [101]:
null_value_imputer(df_exchange_rates)

Null Values after imputation:
 Date        0
Currency    0
Exchange    0
dtype: int64
Date        0
Currency    0
Exchange    0
dtype: int64


In [102]:
df_exchange_rates.dtypes

Date         object
Currency     object
Exchange    float64
dtype: object

In [103]:
df_exchange_rates.head()

,Date,Currency,Exchange
0,1/1/2015,USD,1.0000
1,1/1/2015,CAD,1.1583
2,1/1/2015,AUD,1.2214
3,1/1/2015,EUR,0.8237
4,1/1/2015,GBP,0.6415


In [104]:
df_exchange_rates['Date'] = pd.to_datetime(df_exchange_rates['Date'], errors='coerce')
df_exchange_rates['Date'] = df_exchange_rates['Date'].dt.strftime('%Y-%m-%d')

In [105]:
df_exchange_rates.head()

,Date,Currency,Exchange
0,2015-01-01,USD,1.0000
1,2015-01-01,CAD,1.1583
2,2015-01-01,AUD,1.2214
3,2015-01-01,EUR,0.8237
4,2015-01-01,GBP,0.6415


In [106]:
df_exchange_rates.to_csv('exchange_rates_data.csv', index=False)

In [107]:
import pymysql

In [108]:
myconnection = pymysql.connect(host='127.0.0.1',user='root',password='root')

In [109]:
a=",".join(f"{i} {j}"
for i,j in zip(df_exchange_rates.columns,df_exchange_rates.dtypes)).replace("float64","float").replace("category","text").replace("int64","int").replace("object","text")

In [110]:
f"create table exchange_rates_data ({a})"

'create table exchange_rates_data (Date text,Currency text,Exchange float)'

In [111]:
myconnection.cursor().execute(f"create table data.exchange_rates_data ({a})")

0

In [112]:
len(df_exchange_rates)

11215

In [113]:
sql = "insert into data.exchange_rates_data values"
for i in range(len(df_exchange_rates)):
    myconnection.cursor().execute(f"{sql} {tuple(df_exchange_rates.iloc[i])}")
    myconnection.commit()

In [114]:
df_exchange_rates.head()

,Date,Currency,Exchange
0,2015-01-01,USD,1.0000
1,2015-01-01,CAD,1.1583
2,2015-01-01,AUD,1.2214
3,2015-01-01,EUR,0.8237
4,2015-01-01,GBP,0.6415
